In [ ]:
# Download the Donut repository and the SROIE dataset
# This script is used to fine-tune the Donut model on the SROIE dataset
# This script is a copy of orginal Donut script which runs on google colab
!git clone https://github.com/clovaai/donut.git
!git clone https://github.com/zzzDavid/ICDAR-2019-SROIE

Cloning into 'donut'...
remote: Enumerating objects: 268, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 268 (delta 0), reused 3 (delta 0), pack-reused 264
Receiving objects: 100% (268/268), 62.76 MiB | 35.66 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
# set up the environment
!cd donut && pip install .
!pip install transformers==4.25.1
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/donut
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.3 MB/s e

In [ ]:
import os
import json
import shutil
from tqdm.notebook import tqdm
lines = []
images = []
for ann in tqdm(os.listdir("/content/ICDAR-2019-SROIE/data/key")[500:600]):
    if ann != ".ipynb_checkpoints":
        with open("/content/ICDAR-2019-SROIE/data/key/" + ann) as f:
            data = json.load(f)
    images.append(ann[:-4] + "jpg")
    line = {"gt_parse": data}
    lines.append(line)
with open("./sroie_donut/validation/metadata.jsonl", 'w') as f:
    for i, gt_parse in enumerate(lines):
        line = {"file_name": images[i], "ground_truth": json.dumps(gt_parse)}
        f.write(json.dumps(line) + "\n")
        shutil.copyfile("/content/ICDAR-2019-SROIE/data/img/" + images[i], "./sroie_donut/validation/" + images[i])

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
# make a copy of the template config file
!cp /content/donut/config/train_cord.yaml /content/donut/config/train_sorie.yaml

In [ ]:
# Used the modify the config file to fine-tune the Donut model on the SROIE dataset
# You can find the config file in the train_sorie.yaml file
!cd donut && python train.py --config /content/donut/config/train_sorie.yaml

In [ ]:
!nvidia-smi

Mon Mar 13 13:21:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# a gradio interface to test the model
from donut import DonutModel
from PIL import Image
import torch
model = DonutModel.from_pretrained("/content/donut/result/train_sorie/20230313_111731")
if torch.cuda.is_available():
    model.half()
    device = torch.device("cuda")
    model.to(device)
else:
    model.encoder.to(torch.bfloat16)
model.eval()
image = Image.open("/content/IMG_3517.jpg").convert("RGB")
output = model.inference(image=image, prompt="<s_sroie_donut>")
output

{'predictions': [{'company': 'PHO 16',
   'date': '01/12/2018',
   'address': '20 RULE SCHEFFER',
   'total': '51,00'}]}

In [ ]:
# mount the google drive to save the model
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/donut/result/train_sorie/20230313_111731 /content/drive/MyDrive

Mounted at /content/drive
